In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Goal :**

To build a model that, given the metadata of a browsing session and the sequence of visited pages, predicts which product pages the user will visit next.

🛒 A recommendation engine for an e-commerce rental site

User clicked products in a session →
Predict next products they will visit →
Top 6 products

In [1]:
import pandas as pd
import numpy as np

## 📦 Role of Each Dataset

| Type                        | Dataset                      | What it Contains                                              | How It’s Used in the Project                               |
| --------------------------- | ---------------------------- | ------------------------------------------------------------- | ---------------------------------------------------------- |
| 👤 **User Browsing**        | `metrika_hits`               | Page views, product clicks, device & traffic data             | Build user activity sequences, understand product interest |
| 🧭 **Session Info**         | `metrika_visits`             | Session-level metadata (new user, traffic source, device)     | Create session features and user context                   |
| 🛍️ **Product Info**        | `new_site_products`          | Product categories, brand, price, attributes, age suitability | Build item features for content-based & hybrid models      |
| 🔥 **Popularity**           | `new_site_orders`            | Purchase history, product demand, revenue                     | Calculate product popularity, trending items               |
| 🧠 **High Intent Behavior** | `old_site_carts`             | Cart additions, near-purchase actions                         | Strong purchase intent signals                             |
| 📈 **Extra Behavior**       | `old_site_orders`            | Historical purchase patterns                                  | Learn long-term user preferences                           |
| 🧩 **Similarity Learning**  | `old_site_products`          | Old catalog product attributes                                | Build item similarity embeddings                           |
| 🔗 **ID Bridge**            | `old_site_new_site_products` | Mapping old product IDs to new ones                           | Merge old & new product history                            |


In [2]:
metrika_hits=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_hits.csv")
metrika_hits_test=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_hits_test.csv")
metrika_visits=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_visits.csv")
metrika_visits_test=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_visits_test.csv")
new_site_orders=pd.read_csv("/kaggle/input/rental-product-recommendation-system/new_site_orders.csv")
new_site_products=pd.read_csv("/kaggle/input/rental-product-recommendation-system/new_site_products.csv")
old_site_carts=pd.read_csv("/kaggle/input/rental-product-recommendation-system/old_site_carts.csv")
old_site_new_site_product=pd.read_csv("/kaggle/input/rental-product-recommendation-system/old_site_new_site_products.csv")
old_site_orders=pd.read_csv("/kaggle/input/rental-product-recommendation-system/old_site_orders.csv")
old_site_products=pd.read_csv("/kaggle/input/rental-product-recommendation-system/old_site_products.csv")

/tmp/ipykernel_55/4186222877.py:1: DtypeWarning: Columns (1,3,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  metrika_hits=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_hits.csv")
/tmp/ipykernel_55/4186222877.py:3: DtypeWarning: Columns (1,4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  metrika_visits=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_visits.csv")


In [3]:
print("Metrika hits",metrika_hits.shape)
print("Metrika hits test",metrika_hits_test.shape)
print("Metrika visit",metrika_visits.shape)
print("Metrika visit test",metrika_visits_test.shape)
print("new site orders",new_site_orders.shape)
print("new site products",new_site_products.shape)
print("old site carts",old_site_carts.shape)
print("old site  new site products",old_site_new_site_product.shape)
print("old site orders",old_site_orders.shape)
print("old site products",old_site_products.shape)

Metrika hits (1757619, 31)
Metrika hits test (6880, 31)
Metrika visit (345214, 24)
Metrika visit test (1363, 24)
new site orders (2857, 21)
new site products (688, 97)
old site carts (20839, 13)
old site  new site products (464, 2)
old site orders (17905, 15)
old site products (761, 74)


In [4]:
print("Metrika hits",metrika_hits.columns)

print("Metrika visit",metrika_visits.columns)

print("new site orders",new_site_orders.columns)
print("new site products",new_site_products.columns)
print("old site carts",old_site_carts.columns)
print("old site  new site products",old_site_new_site_product.columns)
print("old site orders",old_site_orders.columns)
print("old site products",old_site_products.columns)

Metrika hits Index(['project_id', 'watch_id', 'date_time', 'client_id', 'region_country',
       'region_city', 'url', 'page_type', 'slug', 'artificial', 'is_page_view',
       'not_bounce', 'link', 'download', 'last_traffic_source',
       'last_search_engine_root', 'last_search_engine', 'last_adv_engine',
       'last_social_network', 'last_social_network_profile',
       'recommendation_system', 'messenger', 'device_category', 'mobile_phone',
       'mobile_phone_model', 'operating_system_root', 'operating_system',
       'browser', 'browser_major_version', 'screen_width', 'screen_height'],
      dtype='object')
Metrika visit Index(['project_id', 'visit_id', 'date_time', 'is_new_user', 'client_id',
       'region_country', 'region_city', 'watch_ids', 'traffic_source',
       'adv_engine', 'search_engine_root', 'search_engine', 'social_network',
       'recommendation_system', 'messenger', 'device_category', 'mobile_phone',
       'mobile_phone_model', 'operating_system_root', 'opera

In [5]:
new_site_products

,id,name,brand,main_category,categories,site_category_paths,color,price_per_period_day,price_per_period2days,price_per_period3days,...,param_measurement_step,param_folded_size,param_unfolded_size,param_backrest_height_adjustment,param_table_positions,param_footrest_positions,param_footrest_adjustment,param_slide_length,param_melodies_count,param_vk_product_category
0,463480210,Кокон для новорожденных Cocoonababy Red Castle...,Red Castle,Коконы для новорожденных,"[""Коконы для новорожденных""]",Все товары ## Все товары/Товары для новорожден...,NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],"[""Детские товары/Детские коляски""]"
1,463480211,Кокон для новорожденных Matello Cocon Baby Lux...,Matello,Коконы для новорожденных,"[""Коконы для новорожденных""]",Все товары ## Все товары/Коконы для новорожденных,NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],"[""Детские товары/Детские коляски""]"
2,463480212,Кокон-люлька для новорожденных Farla Baby Shel...,Farla,Коконы для новорожденных,"[""Коконы для новорожденных""]",Все товары ## Все товары/Товары для новорожден...,NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],"[""Детские товары/Детские коляски""]"
3,463480213,Качели-шезлонг 4moms MamaRoo 4.0 напрокат,4moms,Электрокачели,"[""Качели, шезлонги"",""Электрокачели""]",Все товары ## Все товары/Товары для новорожден...,NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],"[""Детские товары/Детские коляски""]"
4,463480214,Качели-шезлонг 4moms MamaRoo 3.0 напрокат,4moms,Электрокачели,"[""Качели, шезлонги"",""Электрокачели""]","Все товары ## Все товары/Качели, шезлонги ## В...",NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],"[""Детские товары/Детские коляски""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,1651502137,Адаптеры для автолюльки Yoyo,BabyZen,Коляски,"[""Коляски""]",Все товары/Коляски ## Все товары/Коляски/Аксес...,NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],[]
684,1655011689,Электрокачели MamaRoo NEW + весы Maman напрокат,Maman##4moms,Акции недели,"[""Акции недели"",""Качели, шезлонги"",""Весы""]",Все товары ## Все товары/Акции недели ## Все т...,NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],[]
685,1677458905,Обезьянка на кольцах напрокат,Bright Starts,Музыкальные игрушки,"[""Развивающие игрушки"",""Музыкальные игрушки""]",Все товары ## Все товары/Развивающие игрушки #...,NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],[]
686,1677460825,"Развивающая игрушка ""Маленькие друзья"" напрокат",Fisher-Price,Музыкальные игрушки,"[""Развивающие игрушки"",""Музыкальные игрушки""]",Все товары ## Все товары/Развивающие игрушки #...,NaN,NaN,NaN,NaN,...,[],[],[],[],[],[],[],[],[],[]


In [6]:
metrika_hits

,project_id,watch_id,date_time,client_id,region_country,region_city,url,page_type,slug,artificial,...,messenger,device_category,mobile_phone,mobile_phone_model,operating_system_root,operating_system,browser,browser_major_version,screen_width,screen_height
0,1,8194662451870622352,2021-11-24T00:06:51,0203466395347938108,Russia,Moscow,https://oldsite.info/igrovoy-derevyannyy-stoli...,PRODUCT,igrovoy-derevyannyy-stolik-elc-bolshoy,0,...,NaN,2,xiaomi,MI 11,android,android_11,chromemobile,96,412,915
1,1,7945031156395861140,2021-11-24T00:09:06,98977237931745921755,Russia,NaN,https://oldsite.info/?fbclid=PAAabmh91Sd00ZtFJ...,MAIN,NaN,0,...,NaN,2,oppo,CPH1803,android,android_oreo81,android_browser,8,360,760
2,1,6365098314568244974,2021-11-24T00:09:07,98977237931745921755,Russia,NaN,https://oldsite.info/?fbclid=PAAabmh91Sd00ZtFJ...,MAIN,NaN,1,...,NaN,2,oppo,CPH1803,android,android_oreo81,android_browser,8,360,760
3,1,5280229437160873424,2021-11-24T00:09:21,98977237931745921755,Russia,NaN,https://oldsite.info/?fbclid=PAAabmh91Sd00ZtFJ...,MAIN,NaN,1,...,NaN,2,oppo,CPH1803,android,android_oreo81,android_browser,8,360,760
4,1,133574988465249163,2021-11-24T00:09:52,1507358969798591332,Russia,Saint Petersburg,https://oldsite.info/,MAIN,NaN,1,...,NaN,1,NaN,NaN,gnu_linux,linux,chrome,95,1920,1080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757614,0,2825584559588331111,2025-10-14T23:53:35,9910101217269060972,Russia,Moscow,https://newsite.info/collection/kolyaski,CATEGORY,kolyaski,1,...,NaN,2,apple,iPhone,ios_double,ios18,safari_mobile,18,440,956
1757615,0,5093067904021115436,2025-10-14T23:53:35,9910101217269060972,Russia,Moscow,https://newsite.info/collection/kolyaski,CATEGORY,kolyaski,0,...,NaN,2,apple,iPhone,ios_double,ios18,safari_mobile,18,440,956
1757616,0,5531199914965553706,2025-10-14T23:53:35,9910101217269060972,Russia,Moscow,https://newsite.info/collection/kolyaski,CATEGORY,kolyaski,1,...,NaN,2,apple,iPhone,ios_double,ios18,safari_mobile,18,440,956
1757617,0,4996877742628913121,2025-10-14T23:53:36,9910101217269060972,Russia,Moscow,https://newsite.info/collection/kolyaski,CATEGORY,kolyaski,1,...,NaN,2,apple,iPhone,ios_double,ios18,safari_mobile,18,440,956


**Feautre Blocks**

| Component             | Comes From                                       |
| --------------------- | ------------------------------------------------ |
| **User behavior**     | metrika_hits, metrika_visits                     |
| **Product knowledge** | new_site_products, old_site_products             |
| **Purchase intent**   | new_site_orders, old_site_orders, old_site_carts |


**We combine these → create features → train ranking model.**

# **Step 1 : Prepare Behavior Data**

In [8]:
hits=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_hits.csv")
visit=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_visits.csv")
products=pd.read_csv("/kaggle/input/rental-product-recommendation-system/new_site_products.csv")
orders=pd.read_csv("/kaggle/input/rental-product-recommendation-system/new_site_orders.csv")


/tmp/ipykernel_55/1060669359.py:1: DtypeWarning: Columns (1,3,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  hits=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_hits.csv")
/tmp/ipykernel_55/1060669359.py:2: DtypeWarning: Columns (1,4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  visit=pd.read_csv("/kaggle/input/rental-product-recommendation-system/metrika_visits.csv")


# **Step 2 : Build USER SESSIONS (Behavior Base)**

Goal: Understand what each user looked at.

In [ ]:
hits['date_time'] = pd.to_datetime(hits['date_time'])
hits = hits.sort_values(['client_id','date_time'])